Topic: Challenge Set 1 Subject: Explore MTA turnstile data Date: 04/13/2018 Name: student name Worked with: other students' name Location: sea18_ds10/student_submissions/challenges/01-mta/shaikh_reshama/challenge_set_1_reshama.ipynb

In [301]:
import sys

In [302]:
# imports a library 'pandas', names it as 'pd'
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import Image

import pprint

# enables inline plots, without it plots don't show up in the notebook
%matplotlib inline

import dateutil.parser
from datetime import *

In [303]:
# various options in pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)
pd.set_option('display.precision', 3)

In [304]:
#read data from this folder
df1 = pd.read_csv('turnstile_171007.csv')
df2 = pd.read_csv('turnstile_170902.csv')
df3 = pd.read_csv('turnstile_171021.csv')

In [305]:
#Work with turnstile name as combo
def Col_Con(mta):
    mta['TURNSTILE'] = mta['C/A'] + mta['UNIT'] + mta['SCP']
    mta.drop('C/A', axis=1, inplace = True)
    mta.drop('UNIT', axis=1,inplace = True)
    mta.drop('SCP', axis=1, inplace = True)
    mta = mta.drop('LINENAME', axis=1)
    mta = mta.drop('DIVISION', axis=1)
    mta = mta.drop('DESC', axis=1)
    mta.columns = mta.columns.str.strip()
    return mta

In [306]:
mta1 = Col_Con(df1)
mta2 = Col_Con(df2)
mta3 = Col_Con(df3)

In [307]:
leng = len(mta1.index)

In [308]:
# make new columns, blank
#Only use if we are using turnstiles as data 'mta['Cu_ENT'] = [0] * leng'
def new_cols(mta):
    leng = len(mta.index)
    mta['ENT_COUNT'] = [0] * leng
    mta['EXT_COUNT'] = [0] * leng
    mta['DATE_TIME'] = [''] * leng
    return mta

In [309]:
mta1 = new_cols(mta1)
mta2 = new_cols(mta2)
mta3 = new_cols(mta3)

In [310]:
# Fill empty date time column with formula
# I dont think we will use this...
def date_time(df):
    week   = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday','Sunday']
    for i in range(leng):
        datetime = df.DATE[i]+ ' ' + df.TIME[i]      
        value = dateutil.parser.parse(datetime)
        df.at[i,'DATE_TIME'] = value.weekday()
    return df

In [311]:
mta1 = date_time(mta1)
mta2 = date_time(mta2)
mta3 = date_time(mta3)

In [312]:
#fill empty ent_count with formula
def coun_ent(mta):
    for i in range(leng):
        if i == 0: 
            mta.at[i,'ENT_COUNT'] = 0
        else:
            if mta.TURNSTILE[i] == mta.TURNSTILE[i-1]:
                diff = mta.ENTRIES[i] - mta.ENTRIES[i-1]
                # ignore any negative entry count or very large entry counts, turnstile malfunction
                # we should discuss a better 
                if diff < 0: diff = 0
                if diff > 10000: diff = 0
                mta.at[i,'ENT_COUNT'] = diff
            else:
                mta.at[i,'ENT_COUNT'] = 0
    return mta

In [313]:
mta1 = coun_ent(mta1)
mta2 = coun_ent(mta2)
mta3 = coun_ent(mta3)

In [314]:
#fill in new exit counts
def coun_ext(mta):
    for i in range(leng):
        if i == 0: 
            mta.at[i,'EXT_COUNT'] = 0
        else:
            if mta.TURNSTILE[i] == mta.TURNSTILE[i-1]:
                diff = mta.EXITS[i] - mta.EXITS[i-1]
                # ignore any negative entry count or very large entry counts, turnstile malfunction
                # we should discuss a better 
                if diff < 0: diff = 0
                if diff > 10000: diff = 0
                mta.at[i,'EXT_COUNT'] = diff
            else:
                mta.at[i,'EXT_COUNT'] = 0
                
    return mta

In [315]:
mta1 = coun_ext(mta1)
mta2 = coun_ext(mta2)
mta3 = coun_ext(mta3)

In [316]:
def mor_cln(mta):   
    mta = mta.drop('TIME', axis=1)
    mta = mta.drop('ENTRIES', axis=1)
    mta = mta.drop('EXITS', axis=1)
    return mta

In [317]:
mta1 = mor_cln(mta1)
mta2 = mor_cln(mta2)
mta3 = mor_cln(mta3)

In [318]:
# Check all negative and large numbers have been removed

mta1.describe()
mta2.describe()
mta3.describe()

ENT_COUNT   EXT_COUNT
count  197912.000  197912.000
mean      183.690     139.994
std       274.758     237.477
min         0.000       0.000
25%         7.000       6.000
50%        73.000      51.000
75%       252.000     171.000
max      8934.000    4367.000

In [319]:
#tur_uni = mta.TURNSTILE.unique()

In [320]:
"""# Find some time of day trend?
turn = ''
date = ''
sum_ent = 0
cu_list = []
for i in range(leng):
    #Initialing complex if, goal is to have the cumulative for each day 
    if turn != mta.TURNSTILE[i] and i==0:
        turn = mta.TURNSTILE[i]
        date = mta.DATE[i]
        beg_in = i
        sum_ent= 0
    #if the turnstile and date are the same, sum up the ent_count
    elif (turn == mta.TURNSTILE[i] and date == mta.DATE[i]):
        sum_ent += mta['ENT_COUNT'][i]
    #if either the turnstile number changes, or the date changes, dump the sum of entries, 
    #and store in a new column across multpile indexes
    elif turn != mta.TURNSTILE[i] or date != df.DATE[i]:
        for j in range(i-beg_in):
            j +=beg_in
            mta.at[j, 'Cu_ENT'] = sum_ent 
        turn = mta.TURNSTILE[i]
        date = mta.DATE[i]
        beg_in = i
        sum_ent = mta['ENT_COUNT'][i]
mta"""

"# Find some time of day trend?\nturn = ''\ndate = ''\nsum_ent = 0\ncu_list = []\nfor i in range(leng):\n    #Initialing complex if, goal is to have the cumulative for each day \n    if turn != mta.TURNSTILE[i] and i==0:\n        turn = mta.TURNSTILE[i]\n        date = mta.DATE[i]\n        beg_in = i\n        sum_ent= 0\n    #if the turnstile and date are the same, sum up the ent_count\n    elif (turn == mta.TURNSTILE[i] and date == mta.DATE[i]):\n        sum_ent += mta['ENT_COUNT'][i]\n    #if either the turnstile number changes, or the date changes, dump the sum of entries, \n    #and store in a new column across multpile indexes\n    elif turn != mta.TURNSTILE[i] or date != df.DATE[i]:\n        for j in range(i-beg_in):\n            j +=beg_in\n            mta.at[j, 'Cu_ENT'] = sum_ent \n        turn = mta.TURNSTILE[i]\n        date = mta.DATE[i]\n        beg_in = i\n        sum_ent = mta['ENT_COUNT'][i]\nmta"

In [321]:
def merge_agg(ent_agg, ext_agg):
    ent_agg.columns = ent_agg.columns.droplevel(level=1)
    ext_agg.columns = ext_agg.columns.droplevel(level=1)
    #Creating a new data frame removing repeted values
    q_mta = pd.merge(ent_agg, ext_agg, on=['STATION','DATE_TIME'], how='left')
    return q_mta

In [322]:
def get_agg(mta):
    mt = mta.groupby(['STATION','DATE_TIME'])
    ent_agg = mt.agg({'ENT_COUNT':['sum']})
    ent_agg.reset_index(inplace=True)
    ext_agg = mt.agg({'EXT_COUNT':['sum']})
    #returns "index" as columns .reset_index()
    ext_agg.reset_index(inplace=True)
    fin_agg = merge_agg(ent_agg, ext_agg)
    return fin_agg

In [323]:
def get_agg_mean(mta):
    mt = mta.groupby(['STATION','DATE_TIME'])
    ent_agg = mt.agg({'ENT_COUNT':['mean']})
    ent_agg.reset_index(inplace=True)
    ext_agg = mt.agg({'EXT_COUNT':['mean']})
    #returns "index" as columns .reset_index()
    ext_agg.reset_index(inplace=True)
    fin_agg = merge_agg(ent_agg, ext_agg)
    return fin_agg

In [324]:
mta1 = get_agg(mta1)
mta2 = get_agg(mta2)
mta3 = get_agg(mta3)

In [325]:
def super_merge(a,b,c):
    new_df = pd.concat([a, b])
    q_mta = pd.concat([new_df, c])
    q_mta=get_agg_mean(q_mta)
    return q_mta
#, on=['STATION','DATE_TIME'], how='left'
#, on=['STATION','DATE_TIME'], how='left')

In [326]:
super_df = super_merge(mta1,mta2,mta3)
super_df

STATION           DATE_TIME  ENT_COUNT  EXT_COUNT
0           1 AV 2017-08-26 00:00:00          0          0
1           1 AV 2017-08-26 04:00:00       1425        999
2           1 AV 2017-08-26 08:00:00        529        949
3           1 AV 2017-08-26 12:00:00       2208       2516
4           1 AV 2017-08-26 16:00:00       4076       3723
5           1 AV 2017-08-26 20:00:00       3659       4603
6           1 AV 2017-08-27 00:00:00       3282       3589
7           1 AV 2017-08-27 04:00:00       1436       1263
8           1 AV 2017-08-27 08:00:00        386        637
9           1 AV 2017-08-27 12:00:00       1758       1977
10          1 AV 2017-08-27 16:00:00       2750       3922
11          1 AV 2017-08-27 20:00:00       3709       4188
12          1 AV 2017-08-28 00:00:00       2427       2387
13          1 AV 2017-08-28 04:00:00        405        339
14          1 AV 2017-08-28 08:00:00       1219       2634
15          1 AV 2017-08-28 12:00:00       4183       4305
16          1 AV 2017-08-28 16:00:00       2946       3289
17          1 AV 2017-08-28 20:00:00       5660       5308
18          1 AV 2017-08-29 00:00:00       2941       2669
19          1 AV 2017-08-29 04:00:00        497        348
20          1 AV 2017-08-29 08:00:00       1292       2826
21          1 AV 2017-08-29 12:00:00       4430       4405
22          1 AV 2017-08-29 16:00:00       3840       3813
23          1 AV 2017-08-29 20:00:00       5255       5806
24          1 AV 2017-08-30 00:00:00       2944       2979
25          1 AV 2017-08-30 04:00:00        527        352
26          1 AV 2017-08-30 08:00:00       1324       2810
27          1 AV 2017-08-30 12:00:00       4507       4482
28          1 AV 2017-08-30 16:00:00       3740       3596
29          1 AV 2017-08-30 20:00:00       5963       5962
30          1 AV 2017-08-31 00:00:00       3679       3162
31          1 AV 2017-08-31 04:00:00        724        393
32          1 AV 2017-08-31 08:00:00       1225       2858
33          1 AV 2017-08-31 12:00:00       4460       4360
34          1 AV 2017-08-31 16:00:00       3957       3810
35          1 AV 2017-08-31 20:00:00       5801       5661
36          1 AV 2017-09-01 00:00:00       3692       3018
37          1 AV 2017-09-01 04:00:00        745        491
38          1 AV 2017-09-01 08:00:00       1125       2745
39          1 AV 2017-09-01 12:00:00       4185       4886
40          1 AV 2017-09-01 16:00:00       4961       4479
41          1 AV 2017-09-01 20:00:00       5835       5286
42          1 AV 2017-09-30 00:00:00          0          0
43          1 AV 2017-09-30 04:00:00       1687       1183
44          1 AV 2017-09-30 08:00:00        495        845
45          1 AV 2017-09-30 12:00:00       2201       2652
46          1 AV 2017-09-30 16:00:00       3886       4174
47          1 AV 2017-09-30 20:00:00       4614       5410
48          1 AV 2017-10-01 00:00:00       3958       4251
49          1 AV 2017-10-01 04:00:00       1419       1137
50          1 AV 2017-10-01 08:00:00        399        629
51          1 AV 2017-10-01 12:00:00       1835       2397
52          1 AV 2017-10-01 16:00:00       3367       3915
53          1 AV 2017-10-01 20:00:00       3837       4326
54          1 AV 2017-10-02 00:00:00       2440       2247
55          1 AV 2017-10-02 04:00:00        403        305
56          1 AV 2017-10-02 08:00:00       1642       3617
57          1 AV 2017-10-02 12:00:00       4633       5891
58          1 AV 2017-10-02 16:00:00       4775       3878
59          1 AV 2017-10-02 20:00:00       6818       5902
60          1 AV 2017-10-03 00:00:00       2928       2681
61          1 AV 2017-10-03 04:00:00        449        335
62          1 AV 2017-10-03 08:00:00       1615       3778
63          1 AV 2017-10-03 12:00:00       4834       5764
64          1 AV 2017-10-03 16:00:00       4880       3813
65          1 AV 2017-10-03 20:00:00       7226       6361
66          1 AV 2017-10-04 00:00:00       3340       31

In [ ]:
super_df.sort_values(by=['ENT_COUNT'], ascending = 0, inplace = True)
uni_stations = list(super_df.STATION.unique())[:30]
leng = len(super_df.index)
est_donors = []
for i in range(leng):
    est_donors.append(round(super_df['ENT_COUNT'][i]*.0005,2))
super_df['Donor Est.'] = est_donors

In [330]:
uni_df = super_df[super_df['STATION'].isin(uni_stations)]
uni_dt = super_df.DATE_TIME.unique()
print(uni_df)
uni_dt.sort()
uni_dt

               STATION           DATE_TIME  ENT_COUNT  EXT_COUNT  Donor Est.
10231  34 ST-HERALD SQ 2017-10-17 20:00:00      49595      32266   0.000e+00
10225  34 ST-HERALD SQ 2017-10-16 20:00:00      49471      31518   7.125e-01
10237  34 ST-HERALD SQ 2017-10-18 20:00:00      49289      31243   2.645e-01
10243  34 ST-HERALD SQ 2017-10-19 20:00:00      48513      33482   1.104e+00
10195  34 ST-HERALD SQ 2017-10-04 20:00:00      47830      31056   2.038e+00
10201  34 ST-HERALD SQ 2017-10-05 20:00:00      47797      32246   1.830e+00
10145  34 ST-HERALD SQ 2017-08-29 20:00:00      47682      32514   1.641e+00
10139  34 ST-HERALD SQ 2017-08-28 20:00:00      46861      29612   7.180e-01
10151  34 ST-HERALD SQ 2017-08-30 20:00:00      46797      32664   1.930e-01
10157  34 ST-HERALD SQ 2017-08-31 20:00:00      46627      30752   8.790e-01
10182  34 ST-HERALD SQ 2017-10-02 20:00:00      46305      30714   1.375e+00
10249  34 ST-HERALD SQ 2017-10-20 20:00:00      45704      35693   1.855e+00

array(['2017-08-26T00:00:00.000000000', '2017-08-26T00:00:37.000000000',
       '2017-08-26T00:01:17.000000000', ...,
       '2017-10-20T23:57:45.000000000', '2017-10-20T23:58:22.000000000',
       '2017-10-20T23:59:07.000000000'], dtype='datetime64[ns]')

In [ ]:
# Next step! Combine days together and analyze what locations are the most popular and one what days
import seaborn as sns
import matplotlib.dates as mdates

# build the figure
#fig, ax = plt.subplots(figsize=(11,12))
#sns.barplot('DATE_TIME', 'ENT_COUNT', data=uni_df, hue='STATION', ax=ax)



In [ ]:
fig, ax = plt.subplots(figsize=(15,12))
sns.pointplot(y="ENT_COUNT", x="DATE_TIME", data=uni_df, join=True, hue='STATION', ax=ax, linestyles='-', order= uni_dt)

#### Challenge 1   
  
- Open up a new IPython notebook
- Download a few MTA turnstile data files
- Open up a file, use csv reader to read it, make a python dict where
  there is a key for each (C/A, UNIT, SCP, STATION). These are the
  first four columns. The value for this key should be a list of
  lists. Each list in the list is the rest of the columns in a
  row. For example, one key-value pair should look like


{    ('A002','R051','02-00-00','LEXINGTON AVE'):
[
['NQR456', 'BMT', '01/03/2015', '03:00:00', 'REGULAR', '0004945474', '0001675324'],
['NQR456', 'BMT', '01/03/2015', '07:00:00', 'REGULAR', '0004945478', '0001675333'],
['NQR456', 'BMT', '01/03/2015', '11:00:00', 'REGULAR', '0004945515', '0001675364'],
...
]
}

#### Challenge 2

- Let's turn this into a time series.

 For each key (basically the control area, unit, device address and
 station of a specific turnstile), have a list again, but let the list
 be comprised of just the point in time and the count of entries.

This basically means keeping only the date, time, and entries fields
in each list. You can convert the date and time into datetime objects
-- That is a python class that represents a point in time. You can
combine the date and time fields into a string and use the
[dateutil](https://labix.org/python-dateutil) module to convert it
into a datetime object. For an example check
[this StackOverflow question](http://stackoverflow.com/questions/23385003/attributeerror-when-using-import-dateutil-and-dateutil-parser-parse-but-no).

Your new dict should look something like

{    ('A002','R051','02-00-00','LEXINGTON AVE'):
[
[datetime.datetime(2013, 3, 2, 3, 0), 3788],
[datetime.datetime(2013, 3, 2, 7, 0), 2585],
[datetime.datetime(2013, 3, 2, 12, 0), 10653],
[datetime.datetime(2013, 3, 2, 17, 0), 11016],
[datetime.datetime(2013, 3, 2, 23, 0), 10666],
[datetime.datetime(2013, 3, 3, 3, 0), 10814],
[datetime.datetime(2013, 3, 3, 7, 0), 10229],
...
],
....
}



#### Challenge 3

- These counts are for every n hours. (What is n?) We want total daily
  entries.

Now make it that we again have the same keys, but now we have a single
value for a single day, which is the total number of passengers that
entered through this turnstile on this day.


#### Challenge 4

We will plot the daily time series for a turnstile.

In ipython notebook, add this to the beginning of your next cell:

    %matplotlib inline

This will make your matplotlib graphs integrate nicely with the
notebook. To plot the time series, import matplotlib with

    import matplotlib.pyplot as plt

Take the list of [(date1, count1), (date2, count2), ...], for the
turnstile and turn it into two lists:
dates and counts. This should plot it:

    plt.figure(figsize=(10,3))
    plt.plot(dates,counts)
    

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

#### Challenge 5

- So far we've been operating on a single turnstile level, let's
  combine turnstiles in the same ControlArea/Unit/Station combo. There
  are some ControlArea/Unit/Station groups that have a single
  turnstile, but most have multiple turnstilea-- same value for the
  C/A, UNIT and STATION columns, different values for the SCP column.

We want to combine the numbers together -- for each
ControlArea/UNIT/STATION combo, for each day, add the counts from each
turnstile belonging to that combo.

#### Challenge 6

Similarly, combine everything in each station, and come up with a time
series of `[(date1, count1),(date2,count2),...]` type of time series
for each STATION, by adding up all the turnstiles in a station.

#### Challenge 7

Plot the time series for a station.

#### Challenge 8

- Make one list of counts for **one** week for one station. Monday's
count, Tuesday's count, etc. so it's a list of 7 counts.
Make the same list for another week, and another week, and another
week.
`plt.plot(week_count_list)` for every `week_count_list` you created
this way. You should get a rainbow plot of weekly commute numbers on
top of each other.

#### Challenge 9

- Over multiple weeks, sum total ridership for each station and sort
  them, so you can find out the stations with the highest traffic
  during the time you investigate

#### Challenge 10

- Make a single list of these total ridership values and plot it with

    plt.hist(total_ridership_counts)

to get an idea about the distribution of total ridership among
different stations.

This should show you that most stations have a small traffic, and the
histogram bins for large traffic volumes have small bars.

*Additional Hint*:

If you want to see which stations take the meat of the traffic, you
can sort the total ridership counts and make a `plt.bar` graph. For
this, you want to have two lists: the indices of each bar, and the
values. The indices can just be `0,1,2,3,...`, so you can do

    indices = range(len(total_ridership_values))
    plt.bar(indices, total_ridership_values)